In [95]:
import cv2
import os
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)
import matplotlib.pylab as plt
from sklearn.linear_model import LinearRegression
import seaborn as sns
import glob
import math
from IPython import get_ipython
get_ipython().run_line_magic('matplotlib', 'inline')
from IPython.display import Image

#cv
import cv2
import math
from PIL import Image
import math
from scipy import ndimage
import argparse
import imutils
import xml.etree.ElementTree as ET

from tqdm import tqdm
#시각화
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
get_ipython().run_line_magic('matplotlib', 'inline')

import matplotlib.image as mpimg
from matplotlib import font_manager, rc
rc('font',family="AppleGothic")
plt.rcParams["font.family"]="AppleGothic" #plt 한글꺠짐
plt.rcParams["font.family"]="Arial" #외국어꺠짐
plt.rcParams['axes.unicode_minus'] = False # 마이너스 부호 출력 설정
plt.rc('figure', figsize=(10,8))

sns.set(font="AppleGothic", 
        rc={"axes.unicode_minus":False},
        style = 'darkgrid')

import shutil

In [2]:
def show(img):
    #사이즈
    plt.figure(figsize = (10,8))
    #xticks/yticks - 눈금표
    plt.xticks([])
    plt.yticks([])
    #코랩에서 안돌아감 주의
    plt.imshow(img, cmap= 'gray')
    plt.show()

def show_img_compar(img_1, img_2 ):
    f, ax = plt.subplots(1, 2, figsize=(10,10))
    ax[0].imshow(img_1)
    ax[1].imshow(img_2)
    ax[0].axis('off') #hide the axis
    ax[1].axis('off')
    f.tight_layout()
    plt.show()

#### DATA EDA

In [94]:
data_path = "C:\\Users\\yukir\\Documents\Monicas_workspace\\Knee\\vgg_dataset\\mask\\val\\4\\"
save_path = "C:\\Users\\yukir\\Documents\\Monicas_workspace\\Knee\\non_use\\mask_vgg_no\\val\\4\\"
img_list = os.listdir(data_path)

n=0
for i in img_list:
    m = i.split('.jpg')[0]
    try:
        k = m.split('_mask')[0].split('_')[-1]
        # print(k)
        if len(k) == 1:
            shutil.move(data_path+i, save_path+i)
            n+=1
    except:
        pass
n

7

In [69]:
data_path = "C:\\Users\\yukir\\Documents\Monicas_workspace\\Knee\\vgg_dataset\\org_preproc\\val\\4\\"
save_path = "C:\\Users\\yukir\\Documents\\Monicas_workspace\\Knee\\non_use\\knee_vgg_no\\val\\4\\"
img_list = os.listdir(data_path)

n=0
for i in img_list:
    m = i.split('.jpg')[0]
    try:
        k = m.split('_')[-1]
        # print(k)
        if len(k) == 1:
            shutil.move(data_path+i, save_path+i)
            n+=1
    except:
        pass

n

7

In [95]:
data_path = 'C:\\Users\\yukir\\Documents\\Monicas_workspace\\Knee\\dataset\\knee_dataset_prepro\\0\\'
mask_path = 'C:\\Users\\yukir\\Documents\\Monicas_workspace\\Knee\\dataset\\mask_dataset\\0\\'
img_list = os.listdir(data_path)
mask_list = os.listdir(data_path)

print(len(img_list))
print(len(img_list)==len(mask_list))

3253
True


In [96]:
data_path = 'C:\\Users\\yukir\\Documents\\Monicas_workspace\\Knee\\dataset\\knee_dataset_prepro\\1\\'
mask_path = 'C:\\Users\\yukir\\Documents\\Monicas_workspace\\Knee\\dataset\\mask_dataset\\1\\'
img_list = os.listdir(data_path)
mask_list = os.listdir(data_path)

print(len(img_list))
print(len(img_list)==len(mask_list))

1495
True


In [97]:
data_path = 'C:\\Users\\yukir\\Documents\\Monicas_workspace\\Knee\\dataset\\knee_dataset_prepro\\2\\'
mask_path = 'C:\\Users\\yukir\\Documents\\Monicas_workspace\\Knee\\dataset\\mask_dataset\\2\\'
img_list = os.listdir(data_path)
mask_list = os.listdir(data_path)

print(len(img_list))
print(len(img_list)==len(mask_list))

2175
True


In [98]:
data_path = 'C:\\Users\\yukir\\Documents\\Monicas_workspace\\Knee\\dataset\\knee_dataset_prepro\\3\\'
mask_path = 'C:\\Users\\yukir\\Documents\\Monicas_workspace\\Knee\\dataset\\mask_dataset\\3\\'
img_list = os.listdir(data_path)
mask_list = os.listdir(data_path)

print(len(img_list))
print(len(img_list)==len(mask_list))

1086
True


In [99]:
data_path = 'C:\\Users\\yukir\\Documents\\Monicas_workspace\\Knee\\dataset\\knee_dataset_prepro\\4\\'
mask_path = 'C:\\Users\\yukir\\Documents\\Monicas_workspace\\Knee\\dataset\\mask_dataset\\4\\'
img_list = os.listdir(data_path)
mask_list = os.listdir(data_path)

print(len(img_list))
print(len(img_list)==len(mask_list))

251
True


#### DOCTOR'S DATASET

In [3]:
data_path = 'C:\\Users\\yukir\\Documents\\Monicas_workspace\\Knee\\dataset\\doctors\\4\\'
save_path = 'C:\\Users\\yukir\\Documents\\Monicas_workspace\\Knee\\new_doctors\\4\\'

img_list = os.listdir(data_path)

In [91]:
for i in tqdm(range(len(img_list))):
    img = cv2.imread(data_path+img_list[i], cv2.IMREAD_COLOR)
    img_ = cv2.imread(data_path+img_list[i], cv2.IMREAD_COLOR)
    name = img_list[i].replace(' ','').replace('.png','.jpg')
    # print(name)


    ###마스킹####
    blur_k = int((img.mean()*0.5)//2)*2+1 
    img = cv2.medianBlur(img, blur_k)

    #threshold 적용을 위해 Lab에서 Grayscale로 이미지 변환 03
    img = cv2.cvtColor(img, cv2.COLOR_Lab2BGR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    #이미지 평균값을 기준으로 이진화 04
    img = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,7,2)
    # show(img)
    #가장 큰 값의 컨투어로 마스크 만들기 05
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    max_cnt = max(contours, key=cv2.contourArea)
    mask = np.zeros(img.shape, dtype=np.uint8)
    cv2.drawContours(mask, [max_cnt], -1, (255,255,255), -1)

    k = cv2.getStructuringElement(cv2.MORPH_RECT, (5,5))
    mask = cv2.dilate(mask,k)
    height, width= mask.shape
    try:

        for y in range(int(height*0.05),height):
            if max(mask[y,int(width*0.3):int(width*0.7)]) > 0:
                start_y = y-int(height*0.05)
                break
        #왼쪽
        for x in range(int(width*0.05),width):
            if max(mask[int(height*0.3):int(height*0.7),x]) > 0:
                start_x = x
                break
        #오른쪽
        for x in range(int(width*0.95),-1,-1):
            if max(mask[int(height*0.3):int(height*0.7),x]) > 0:
                end_x = x-2
                break

        img_ = img_[start_y:,start_x:end_x]

    except:
        print('error')
        pass

    ### preprocessing ###

    hist, bins = np.histogram(img_.flatten(), 256,[0,256])

    cdf = hist.cumsum()

    # cdf의 값이 0인 경우는 mask처리를 하여 계산에서 제외
    # mask처리가 되면 Numpy 계산에서 제외가 됨
    # 아래는 cdf array에서 값이 0인 부분을 mask처리함
    cdf_m = np.ma.masked_equal(cdf,0)

    #History Equalization 공식
    cdf_m = (cdf_m - cdf_m.min())*255/(cdf_m.max()-cdf_m.min())

    # Mask처리를 했던 부분을 다시 0으로 변환
    cdf = np.ma.filled(cdf_m,0).astype('uint8')

    img2 = cdf[img_]

    img3 = cv2.resize(img2, dsize=(224,224))

    # show_img_compar(img, img3)

    cv2.imwrite(save_path+name, img3)

100%|██████████| 206/206 [00:02<00:00, 75.39it/s]


### NEW DATASET

In [172]:
data_path='C:\\Users\\yukir\\Documents\\Monicas_workspace\\Knee\\dataset\\knee_dataset_prepro\\'
path_list = os.listdir(data_path)
for i in path_list:
    k = os.listdir(data_path+i)
    print(len(k))

2496
938
1618
529
1


In [170]:
import random

data_path='C:\\Users\\yukir\\Documents\\Monicas_workspace\\Knee\\dataset\\knee_dataset_prepro\\4\\'
save_path = 'C:\\Users\\yukir\\Documents\\Monicas_workspace\\Knee\\dataset\\new_vgg\\val\\4\\'



for i in range(100):
    try:
        num = random.randint(1, len(os.listdir(data_path)))
        img_list = os.listdir(data_path)
        name = img_list[num]
        shutil.move(data_path+name, save_path+name)
    except:
        continue
    
    if len(os.listdir(save_path))==100:
        break


print(len(os.listdir('C:\\Users\\yukir\\Documents\\Monicas_workspace\\Knee\\dataset\\new_vgg\\val\\4\\')))
    

4


In [124]:
len(os.listdir(data_path))

457